In [200]:
import os
import math
import netCDF4 as nc
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from scipy import interpolate

# Specify current directory and directory with data files
cur_dir = os.getcwd()
data_dir = os.path.join(cur_dir,r'20210205_LinkingIntrinsicAndApparentRelationships')
    
# Name of the Scenario 1 netCDF Source File
    
s2_source_file = 'Scenario2SourceFile.nc'

# Assign the data to the proper variable and assign names; most of the
# variables have latitude, longitude, month, and depth; for Scenario 1
# we are just using the surface values where depth=1

data = nc.Dataset(os.path.join(data_dir,s2_source_file))
lat = data['YT_OCEAN'][:]
lon = data['XT_OCEAN'][:]
po4 = data['PO4'][:]
po4 = np.squeeze(po4)
fed = data['FED'][:]
fed = np.squeeze(fed)
irr_mix = data['IRR_MIX'][:]
irr_mix = np.squeeze(irr_mix)

time_days = [i for i in range(1,366)] # Time vector for days in year
time_hours = [i for i in np.linspace(1,366,365*24)] # Time vector for hours in year

# Creates hourly interpolations for phosphate and iron from the daily
# dataset for each lat/lon pair; this gives matrices for phosphate and iron
# that contain hourly values for each lat/lon pair over the course of a
# year

po4_int = np.empty((8760,80,120))
po4_int[:] = np.nan
fed_int = np.empty((8760,80,120))
fed_int[:] = np.nan

for i in range(len(lon)):
    for j in range(len(lat)):
        po4_int_func = interpolate.interp1d(time_days,po4[:,j,i],fill_value="extrapolate") # Function for interpolating phosphate values
        po4_int[:,j,i] = po4_int_func(time_hours) # Interpolates phosphate values for hours
        fed_int_func = interpolate.interp1d(time_days,fed[:,j,i],fill_value="extrapolate") # Function for interpolating iron values
        fed_int[:,j,i] = fed_int_func(time_hours) # Interpolates iron values for hours
        
delta = np.empty(len(time_days))
for i in range(len(time_days)):
    delta[i] = -23.5*np.cos(np.deg2rad(360/365*time_days[i]+10))
    
lod = np.empty((len(lat),len(delta)))
for i in range(len(lat)):
    for j in range(len(delta)):
        if -np.tan(np.deg2rad(lat[i]))*np.tan(np.deg2rad(delta[j])) < -1:
            lod[i,j] = 24
        elif -np.tan(np.deg2rad(lat[i]))*np.tan(np.deg2rad(delta[j])) > 1:
            lod[i,j] = 0
        else:
            lod[i,j] = np.rad2deg(np.arccos(-np.tan(np.deg2rad(lat[i]))*np.tan(np.deg2rad(delta[j]))))*(2/15)

In [ ]:


# Calculate the limitation terms based on the inputs from phosphate, iron, and light

pstar = 1.9e-6

phoslim = po4/(1e-7+po4)
ironlim = fed/(2e-10+fed)
lightlim = irr_mix/(34.3+irr_mix)

pl_size = phoslim.shape
bio = np.empty((12,80,120))
bio[:] = np.nan

for i in range(12):
    for j in range(80):
        for k in range(120):
            if phoslim[i,j,k] < ironlim[i,j,k]:
                bio[i,j,k] = pstar*phoslim[i,j,k]*lightlim[i,j,k]
            elif ironlim[i,j,k] < phoslim[i,j,k]:
                bio[i,j,k] = pstar*ironlim[i,j,k]*lightlim[i,j,k]
                
# Create lists for variable names and units

variables = ['Index','Latitude','Longitude','Month','Macronutrient','Micronutrient','Irradiance','Biomass']
units = ['','degrees E','degrees N','','mol/kg','mol/kg','W/m2','mol/kg']

# Move the values from each matrix into a column corresponding to the correct
# latitude, longitude, and month; creates matrix where each column corresponds
# to a variable and each row corresponds to an observation

index_matrix = range(1,9601)
index_matrix = np.reshape(index_matrix,(120,80))

final = np.empty((120*80*12,8))
final[:] = np.nan

for i in range(12):
    for j in range(80):
        for k in range(120):
            index_number = index_matrix[k,j]
            final[index_number+((i-1)*9600),0] = index_number + ((i-1)*9600)
            final[index_number+((i-1)*9600),1] = lat[j,k]
            final[index_number+((i-1)*9600),2] = lon[j,k]
            final[index_number+((i-1)*9600),3] = time_initial[i]
            final[index_number+((i-1)*9600),4] = po4[i,j,k]
            final[index_number+((i-1)*9600),5] = fed[i,j,k]
            final[index_number+((i-1)*9600),6] = irr_mix[i,j,k]
            final[index_number+((i-1)*9600),7] = bio[i,j,k]
            
final = final[~np.isnan(final).any(axis=1),:]
final = pd.DataFrame(final,columns=variables)

# Specify the response and predictor columns

resp = [7]
pred = [4,5,6]